In [ ]:
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd
import math
from scipy import stats
import matplotlib.pyplot as plt
import random
from pyeasyga.pyeasyga import GeneticAlgorithm

In [ ]:
def mean_pred(y_true, y_pred):
    RMSE = tf.sqrt(tf.losses.mean_squared_error(y_true, y_pred))
    return RMSE

In [ ]:
x_sheet = pd.read_excel("../data/coded_data.xlsx", usecols=[1, 2, 3, 4, 9, 10, 11])
x_sheet = x_sheet.fillna(0)

df = np.asmatrix(x_sheet.values)
y_sheet = pd.read_excel("../data/coded_data.xlsx", usecols=[7])
labels = np.array(y_sheet.values)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df, labels, test_size=0.2, random_state=89
)

In [ ]:
data = X_train
fitnesses = [1]


def fitness(individual, data):
    fitness = 0
    ind_predicted = model.predict(individual)[0]
    fitness = abs((ind_predicted - intercept) / slope)
    # print(fitness)
    if fitnesses[-1] > fitness:
        fitnesses.append(fitness)
    return fitness

In [ ]:
def create_individual(data):
    dmso = random.uniform(0, 0.25)
    sfb = 0.11
    meio_cultura = 0.77
    tipo_meio = 4
    sugar_choice = random.randint(1, 2)
    if sugar_choice == 1:
        trehalose = random.uniform(0, 0.5)
        glic = 0
        sac = 0
    elif sugar_choice == 2:
        glic = random.uniform(0, 0.5)
        trehalose = 0
        sac = 0
    elif sugar_choice == 3:
        sac = random.uniform(0, 0.5)
        trehalose = 0
        glic = 0
    return np.array([[dmso, sfb, meio_cultura, tipo_meio, trehalose, glic, sac]])


print(create_individual(data))

In [ ]:
def crossover(parent_1, parent_2):
    crossover_index = np.random.randint(0, np.shape((parent_1))[1])
    child_1 = np.array([parent_1[0][:crossover_index]])
    child_1 = np.append(child_1, parent_2[0][crossover_index:])
    child_2 = np.array([parent_2[0][:crossover_index]])
    child_2 = np.append(child_2, parent_1[0][crossover_index:])
    child_1 = child_1.reshape(1, num_var)
    child_2 = child_2.reshape(1, num_var)
    return child_1, child_2

In [ ]:
def mutate(individual):
    mutate_index = random.randrange(np.shape(individual)[0])
    if individual[0][mutate_index] == 0:
        individual[0][mutate_index] == 1
    else:
        individual[0][mutate_index] == 0

In [ ]:
pop = 1000
ga = GeneticAlgorithm(
    data,
    population_size=pop,
    generations=150,
    crossover_probability=0.2,
    mutation_probability=0.1,
    elitism=False,
    maximise_fitness=False,
)

In [ ]:
num_var = X_train.shape[1]
model = tf.keras.models.load_model(
    "models/model_current.h5", custom_objects={"mean_pred": mean_pred}
)
test_ans = model.predict(X_test)
slope, intercept, r_value, p_value, std_err = stats.linregress(
    y_test[:, 0], test_ans[:, 0]
)

In [ ]:
ga.create_individual = create_individual
ga.fitness_function = fitness
ga.crossover_function = crossover
ga.mutate_function = mutate

ga.run()
best_ind = ga.best_individual()
print(best_ind)

In [ ]:
x_axis = []
i = 0
for index in range(len(fitnesses)):
    i += (
        1
    )
    x_axis.append(i)
print(x_axis)
print(fitnesses)

In [ ]:
plt.scatter(x_axis, fitnesses)
plt.ylabel("Fitness")
plt.show()
plt.savefig("../images/genetic_algorithm_eval.png")

In [ ]:
dmso = best_ind[1][0][0] * 100
sfb = best_ind[1][0][1] * 100
conc_meio = best_ind[1][0][2] * 100
meio = best_ind[1][0][3]
treh = best_ind[1][0][4] * 100
glic = best_ind[1][0][5] * 100
sac = best_ind[1][0][6] * 100
print(
    " Parametros: \n DMSO:%f \n SFB: %f \n CONCENTRACAO DE MEIO: %f\n MEIO: %f\n Trehalose: %f\n Glicerol %f\n Sacarose: %f\n"
    % (dmso, sfb, conc_meio, meio, treh, glic, sac)
)